    
<h1 style="text-align: center; color: purple;" markdown="1">QTM 340, Q2: Scraping song lyrics url using the Genius API </h1>

<h2 style="text-align: center; color: #012169" markdown="1">Zejia Chen</h2>


In [1]:
# import requests and beautiful soup package
from bs4 import BeautifulSoup
import requests

## Scraping artist and song title on New York Times

In [2]:
# send request to New York times 
resp_nyt = requests.get("https://www.nytimes.com/interactive/2019/08/19/us/politics/presidential-campaign-songs-playlists.html")
html_str_nyt = resp_nyt.text
document_nyt = BeautifulSoup(html_str_nyt, "html.parser")

In [3]:
# Extracting the song artist and titles
song_dict = {}  


for tag in document_nyt.find_all("span", attrs = {"class": "song-title"}):
    title_name = tag.string
    #print(title_name)
    artist_name = tag.find_next_sibling("span", attrs = {"class": "song-artist"}).string
    #print(artist_name)
    song_dict[title_name] = artist_name 
    
song_dict


{'Respect': 'Aretha Franklin',
 '9 to 5': 'Dolly Parton',
 'Stand by Me': 'Ben E. King',
 'Work That': 'Mary J. Blige',
 'High Hopes': 'Panic! at the Disco',
 'Dance to the Music': 'Sly & the Family Stone',
 'Love Train': 'The O’Jays',
 'I’m Coming Out': 'Diana Ross',
 'One Nation Under a Groove': 'Funkadelic',
 'Rocking in the Free World': 'Neil Young',
 'My Shot': 'Lin-Manuel Miranda',
 'Superwoman': 'Alicia Keys',
 'Good as Hell': 'Lizzo',
 'Higher Ground': 'Stevie Wonder',
 'Lovely Day': 'Bill Withers',
 'California Love ': 'Tupac Shakur',
 'Run the World (Girls)': 'Beyoncé',
 'Think': 'Aretha Franklin',
 'Dis Generation': 'A Tribe Called Quest',
 'Girl on Fire': 'Alicia Keys',
 'Revolution': 'Toots and the Maytals',
 'Power to the People': 'John Lennon',
 'Made in the USA': 'Demi Lovato',
 'Believer': 'Imagine Dragons',
 'Latinoamérica': 'Calle 13',
 'God Bless the USA': 'Lee Greenwoood',
 'I Love Rock ’n Roll': 'Joan Jett & the Blackhearts',
 'Confident': 'Demi Lovato',
 'I Wanna

***

## Getting the correspond lyrics from Genius using Genius.api

In [4]:
# getting access to the genuis api
client_access_token = "Ftox8kH6D0QYYWgUvfrh8sJHr-G6DG_gkWLk9Q2h1oTAMs89IVFlFZZcVRzaQ7R7"
search_term = "My Shot"
genius_search_url = f'http://api.genius.com/search?q={search_term}&access_token={client_access_token}'

There are two approaches in my approaches to scrap song url using the Genius API:

The broad search method allows me to retrive all the lyrics url accroding to the candidat song track list.
However, one disadvantage of the method is that the scrapper will scrap multiple urls if a song has multiple versions displayed on Genius.

For example:
The song "I'm coming out" by Diana Ross has four different versions on Genius.
<br>'https://genius.com/Diana-ross-im-coming-out-lyrics' 
<br>'https://genius.com/Diana-ross-im-coming-out-single-version-lyrics' 
<br>'https://genius.com/Diana-ross-im-coming-out-unreleased-chic-mix-lyrics' 
<br>'https://genius.com/Diana-ross-im-coming-out-maurice-joshua-club-mix-lyrics'

The scrapper will include all of them if there were not any extra conditional statments parsed to refine the search.

In [5]:
song_list = []

for title in song_dict.keys():
    searchKey = title
    url_iter = f'http://api.genius.com/search?q={searchKey}&access_token={client_access_token}'
    data = requests.get(url_iter).json()
    for song in data['response']['hits']:
        
        song_artist = song['result']['primary_artist']['name']
        song_title = song['result']['title']
        if song_artist == song_dict[searchKey]:
            song_url = []
            song_list.append(list((song_artist, '\"' + searchKey + '\": ', song_url)))
            song_url.append(song['result']['url'])
            
print(len(song_list))
song_list[:15]

129


[['Aretha Franklin',
  '"Respect": ',
  ['https://genius.com/Aretha-franklin-respect-lyrics']],
 ['Dolly Parton',
  '"9 to 5": ',
  ['https://genius.com/Dolly-parton-9-to-5-lyrics']],
 ['Ben E. King',
  '"Stand by Me": ',
  ['https://genius.com/Ben-e-king-stand-by-me-lyrics']],
 ['Mary J. Blige',
  '"Work That": ',
  ['https://genius.com/Mary-j-blige-work-that-lyrics']],
 ['Panic! at the Disco',
  '"High Hopes": ',
  ['https://genius.com/Panic-at-the-disco-high-hopes-lyrics']],
 ['The O’Jays',
  '"Love Train": ',
  ['https://genius.com/The-ojays-love-train-lyrics']],
 ['Diana Ross',
  '"I’m Coming Out": ',
  ['https://genius.com/Diana-ross-im-coming-out-lyrics']],
 ['Diana Ross',
  '"I’m Coming Out": ',
  ['https://genius.com/Diana-ross-im-coming-out-single-version-lyrics']],
 ['Diana Ross',
  '"I’m Coming Out": ',
  ['https://genius.com/Diana-ross-im-coming-out-unreleased-chic-mix-lyrics']],
 ['Diana Ross',
  '"I’m Coming Out": ',
  ['https://genius.com/Diana-ross-im-coming-out-mauric

The refined search method contain the one extra conditional statment to let the scrapper to scrap the url only if there is a exact match between song titles in the candidate list and the on Genius.

The obvious advantage of this approach is that it will ensure there is only one url corresponding to the song and the artist.
<br>
However, the scrapper is not able to scrap some of the url dur some naming convention discrepancy between the titles in the candidate list and the one on Genius.

For example, <br>
The artist "Toots and the Maytals" shown in the candidate list is displayed as "Toots & the Maytals" on Genius which results in the discrepancy of the output.

In [ ]:
song_list_2 = []
for title in song_dict.keys():
    searchKey = title
    #print('key = ' + searchKey)
    url_iter = f'http://api.genius.com/search?q={searchKey}&access_token={client_access_token}'
    data = requests.get(url_iter).json()
    for song in data['response']['hits']:
        
        song_artist = song['result']['primary_artist']['name']
        song_title = song['result']['title']
        if song_artist == song_dict[searchKey] and song_title == searchKey:
            song_list_2.append(list((song_artist, '\"' + searchKey + '\": ', song['result']['url'])))
            
print(len(song_list_2))
song_list_2[:15]

**Packages used in this document**
<br>
`requests`
<br>
`BeautifulSoup`

In [ ]:
!jupyter nbconvert --to html Scraping_song_lyrics_url_using_the_Genius_API-submit.ipynb

&nbsp;
<hr />
<p style="font-family:palatino; text-align: center;font-size: 15px">QTM 340 Quiz 2 (Scraping song lyrics url using the Genius API)</a></p>
<p style="font-family:palatino; text-align: center;font-size: 15px"> <em> Zejia Chen</em></a></p>
<p style="font-family:palatino; text-align: center;font-size: 15px"><span style="color: #6666FF;"><em>zejia.chen@emory.edu</em></span></p>

<p style="font-family:palatino; text-align: center; color: #012169;font-size: 15px">Emory University</a></p>

&nbsp;